In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv("drive/MyDrive/Practicum/pharmacy_claims.csv.gz")
print(df.shape)
df.head()

<ipython-input-3-edb31e30d49e>:1: DtypeWarning: Columns (14,22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("drive/MyDrive/Practicum/pharmacy_claims.csv.gz")


(2570133, 48)


,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c38,_c39,_c40,_c41,_c42,_c43,_c44,_c45,_c46,_c47
0,351051,99K1,NaN,NaN,GH06,99K1E001,IMD Non-Medigap,RISK,BlueChoice,Maryland,...,30,90.0,0,5,0.00,0.00,0.0,0.0,0.0,0.00
1,351051,99K1,NaN,NaN,GH06,99K1E001,IMD Non-Medigap,RISK,BlueChoice,Maryland,...,-28,-2.0,0,5,-8608.84,-4080.68,-150.0,0.0,-150.0,-3780.68
2,351051,99K1,NaN,NaN,GH06,99K1E001,IMD Non-Medigap,RISK,BlueChoice,Maryland,...,28,6.0,0,0,0.00,0.00,0.0,0.0,0.0,0.00
3,351051,99K1,NaN,NaN,GH06,99K1E001,IMD Non-Medigap,RISK,BlueChoice,Maryland,...,28,6.0,0,0,0.00,0.00,0.0,0.0,0.0,0.00
4,351051,99K1,NaN,NaN,GH06,99K1E001,IMD Non-Medigap,RISK,BlueChoice,Maryland,...,28,6.0,0,0,0.00,0.00,0.0,0.0,0.0,0.00


In [4]:
data_dict = pd.read_excel('drive/MyDrive/Practicum/Data Dictionary Pharmacy Claims.xlsx', skiprows=1)
data_dict.head()

,Field,Definition
0,Member Life ID,Unique Identifier
1,Group Number,The groups are defined based on the account's ...
2,Section,The groups are defined based on the account's ...
3,Department,The groups are defined based on the account's ...
4,Package Code / Class Code,The groups are defined based on the account's ...


In [5]:
col_names = data_dict['Field'].apply(lambda x: (x.lower().split("/")[0].split("(")[0].strip().replace(" ", "_"))).tolist()
df.columns = col_names

In [6]:
df = df[df['member_life_id']!=-1]
df.shape

(1512707, 48)

In [7]:
df = df[df["subgroup"]!='001901088MD3000008K6GW']

In [8]:
cols_to_gen = [
    'member_life_id', 'birth_date', 'gender_code',
    'subscriber_zip', 'claim_number', 'prescription_number',
    'ndc', 'drug_name', 'specialty_drug_indicator', 'generic_indicator',
    'drug_tier', 'fill_date', 'paid_date', 'days_supply', 'quantity_dispensed',
    'refill_number', 'number_of_refills', 'billed_amount', 'paid_amount'
]
df = df[cols_to_gen]

In [9]:
for col in df.columns:
  df[col] = df[col].astype(str)

In [10]:
sample_df = df.sample(100000, random_state=10)

In [11]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [13]:
!pip install be-great

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.7 MB/s eta 0:00:00


In [ ]:
from be_great import GReaT
model = GReaT(llm='distilgpt2', batch_size=16, epochs=1, save_steps=400000)

model.fit(sample_df)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


In [ ]:
synthetic_data = model.sample(n_samples=100000)